In [63]:
# import
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers

# 1. calculate gradient using tf.GradientTape

In [55]:
def f(w):
    y = w**2
    z = 2*y + 5
    return z

In [ ]:
w = tf.Variable(2.0, dtype=tf.float32)

with tf.GradientTape() as tape:
    grad = tape.gradient(f(w), [w])
print(grad)

# Context Manager
 - Context managers allow you to allocate and release resources precisely when you want to. 
The most widely used example of context managers is the 'with' statement.
 - At the very least a context manager has an '__enter__' and '__exit__' method defined.

# Why 'tf.GradientTape' is a context manager?
It is consists of '__enter__' and '__exit__' method.
So using 'tf.GradientTape', you have to use 'with' statement for context manager.

# Reference
https://book.pythontips.com/en/latest/context_managers.html

# Examples

```python
w = tf.Variable(2.0, dtype=tf.float32)

with tf.GradientTape() as tape:
    zz = f(w)
grad = tape.gradient(zz, [w])
print(grad)
```

```python
import time

class Timer:
    def print(self):
        print(f"Elapsed time: {self.end_time - self.start_time:.2f} seconds")
        
    def __enter__(self):
        self.start_time = time.time()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end_time = time.time()
        print(f"Elapsed time: {self.end_time - self.start_time:.2f} seconds")

# Usage:
with Timer() as tm:
    # Some time-consuming operations
    sum(range(1000000))
    
tm.print() # Elapsed time: 0.06 seconds
```

# 2. implementing linear regression with tf.GradientTape

In [57]:
w = tf.Variable(4.0)
b = tf.Variable(1.0)

In [58]:
@tf.function
def h(x):
    return w*x + b

@tf.function
def mse(y_pred, y):
    return tf.reduce_mean((y_pred - y)**2) # tf.reduce_mean : Computes the mean of elements across dimensions of a tensor.

# Decorator
 - The decorator pattern is a software design pattern that allows us to dynamically add functionality to classes without creating subclasses and affecting the behavior of other objects of the same class.
 - By using the decorator pattern, we can easily generate different permutations of functionality that we might want without creating an exponentially increasing number of subclasses, making our code increasingly complex and bloated.
 - https://machinelearningmastery.com/a-gentle-introduction-to-decorators-in-python/

# tf.function
 - tf.function is a decorator function provided by TensorFlow. 
 - It is used to decorate a Python function to let TensorFlow know that it must build a graph out of the function when called, instead of executing the Python code directly.
 - https://www.tensorflow.org/guide/function
 - https://www.tensorflow.org/guide/intro_to_graphs?hl=ko

In [39]:
x_test = [3.5, 5, 5.5, 6]
print(h(x_test))

tf.Tensor([15. 21. 23. 25.], shape=(4,), dtype=float32)


In [59]:
x = [1, 2, 3, 4, 5, 6, 7, 8, 9] 
y = [11, 22, 33, 44, 53, 66, 77, 87, 95] 

optimizer = tf.optimizers.SGD(0.01) # gradient descent with learning rate 0.01

for i in range(301): # 300 epochs for gradient descent
    with tf.GradientTape() as tape:
        y_pred = h(x)
        cost = mse(y_pred, y)
    grads = tape.gradient(target=cost, sources=[w, b])
    # apply_gradients : Update variables by using their associated gradients.
    optimizer.apply_gradients(grads_and_vars=zip(grads, [w, b])) # grads_and_vars = List of (gradient, variable) pairs.
    if i % 30 == 0:
        print("epoch : {:3} | w의 값 : {:5.4f} | b의 값 : {:5.4} | cost : {:5.6f}".format(i, w.numpy(), b.numpy(), cost))

epoch :   0 | w의 값 : 8.2133 | b의 값 : 1.664 | cost : 1402.555542
epoch :  30 | w의 값 : 10.5119 | b의 값 : 1.884 | cost : 1.306967
epoch :  60 | w의 값 : 10.5318 | b의 값 : 1.759 | cost : 1.252898
epoch :  90 | w의 값 : 10.5494 | b의 값 : 1.648 | cost : 1.210658
epoch : 120 | w의 값 : 10.5650 | b의 값 :  1.55 | cost : 1.177665
epoch : 150 | w의 값 : 10.5787 | b의 값 : 1.464 | cost : 1.151890
epoch : 180 | w의 값 : 10.5909 | b의 값 : 1.387 | cost : 1.131752
epoch : 210 | w의 값 : 10.6016 | b의 값 :  1.32 | cost : 1.116026
epoch : 240 | w의 값 : 10.6111 | b의 값 :  1.26 | cost : 1.103736
epoch : 270 | w의 값 : 10.6195 | b의 값 : 1.207 | cost : 1.094144
epoch : 300 | w의 값 : 10.6269 | b의 값 : 1.161 | cost : 1.086645


# 3. Using keras 

In [66]:
x = [1, 2, 3, 4, 5, 6, 7, 8, 9]
y = [11, 22, 33, 44, 53, 66, 77, 87, 95]

model = Sequential()
model.add(Dense(1, input_dim=1, activation='linear'))
model.compile(loss='mse', optimizer=optimizers.legacy.SGD(lr=0.01), metrics=['mse'])
hist = model.fit(x,y,epochs=300, batch_size=1)

Epoch 1/300
9/9 [==============================] - 0s 668us/step - loss: 679.4573 - mse: 679.4573
Epoch 2/300
9/9 [==============================] - 0s 575us/step - loss: 3.2847 - mse: 3.2847
Epoch 3/300
9/9 [==============================] - 0s 594us/step - loss: 3.5417 - mse: 3.5417
Epoch 4/300
9/9 [==============================] - 0s 555us/step - loss: 2.7279 - mse: 2.7279
Epoch 5/300
9/9 [==============================] - 0s 597us/step - loss: 2.5643 - mse: 2.5643
Epoch 6/300
9/9 [==============================] - 0s 626us/step - loss: 4.4502 - mse: 4.4502
Epoch 7/300
9/9 [==============================] - 0s 640us/step - loss: 3.5825 - mse: 3.5825
Epoch 8/300
9/9 [==============================] - 0s 612us/step - loss: 4.1548 - mse: 4.1548
Epoch 9/300
9/9 [==============================] - 0s 624us/step - loss: 2.0442 - mse: 2.0442
Epoch 10/300
9/9 [==============================] - 0s 611us/step - loss: 4.3801 - mse: 4.3801
Epoch 11/300
9/9 [==============================] - 0s 

In [68]:
print(model.predict([7.5])) # input_dims = 1

1/1 [==============================] - 0s 43ms/step
[[80.751884]]
